In [131]:
#pip install pandas

In [132]:
#pip install matplotlib

In [133]:
#pip install scipy

In [134]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import json
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
#BASE URL
BaseURL = 'https://api-web.nhle.com'

In [135]:
# STARTINGCSV2023=pd.read_csv('LastYear2023Stats.csv')  
# STARTINGCSV2024=pd.read_csv('ThisYear2024Stats.csv')  
#STARTINGCSV2025=pd.read_csv('NHL2526seasonplayerstats.csv')  
STARTINGCSV2025=pd.read_csv('NewPlayerStats2526.csv')  

STARTINGCSV=STARTINGCSV2025#pd.concat([STARTINGCSV2023, STARTINGCSV2024])

STARTINGCSV.sort_values(by=['gameId'])
STARTINGCSV[STARTINGCSV['homeRoadFlag']==1]


,playerId,sweaterNumber,name,position,goals,assists,points,plusMinus,pim,hits,powerPlayGoals,sog,faceoffWinningPctg,toi,blockedShots,shifts,giveaways,takeaways,firstName,lastName,Team,gameId,homeRoadFlag
12,8477935,9,{'default': 'S. Bennett'},C,0,0,0,-1,0,1,0,4,0.555556,19:15,0,26,0,1,Sam,Bennett,FLA,2025020001,1
13,8478421,10,{'default': 'A. Greer'},L,1,0,1,1,5,5,0,1,0.000000,07:19,1,13,0,0,A.J.,Greer,FLA,2025020001,1
14,8482713,11,{'default': 'M. Samoskevich'},R,0,2,2,1,0,3,0,0,0.000000,11:18,0,21,0,0,Mackie,Samoskevich,FLA,2025020001,1
15,8479981,12,{'default': 'J. Gadjovich'},L,0,1,1,1,0,4,0,1,0.000000,07:18,1,12,0,1,Jonah,Gadjovich,FLA,2025020001,1
16,8477933,13,{'default': 'S. Reinhart'},C,0,0,0,-1,0,1,0,3,0.333333,18:14,2,23,2,0,Sam,Reinhart,FLA,2025020001,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5431,8475324,3,{'default': 'N. Jensen'},D,0,1,1,3,0,2,0,1,0.000000,18:45,2,22,0,0,Nick,Jensen,OTT,2025020149,1
5432,8484321,33,{'default': 'N. Matinpalo'},D,0,0,0,-2,0,3,0,0,0.000000,17:31,1,28,1,0,Nikolas,Matinpalo,OTT,2025020149,1
5433,8482095,43,{'default': 'T. Kleven'},D,0,0,0,-2,0,0,0,0,0.000000,19:21,4,30,1,0,Tyler,Kleven,OTT,2025020149,1
5434,8478469,72,{'default': 'T. Chabot'},D,0,1,1,3,0,1,0,1,0.000000,20:20,2,26,1,0,Thomas,Chabot,OTT,2025020149,1


In [136]:
# #https://medium.com/@alexandre.tkint/send-pandas-data-frame-via-email-in-python-c346314321fa


# from email.mime.application import MIMEApplication
# from email.mime.multipart import MIMEMultipart
# from email.mime.text import MIMEText
# import smtplib
# def send_email(send_to, subject, df):
#     send_from = "dummy@gmail.com"
#     password = "your_password"
#     message = """\
#     <p><strong>This is a test email&nbsp;</strong></p>
#     <p><br></p>
#     <p><strong>Greetings&nbsp;</strong><br><strong>Alexandre&nbsp;    </strong></p>
#     """
# for receiver in send_to:
#         multipart = MIMEMultipart()
#         multipart["From"] = send_from
#         multipart["To"] = receiver
#         multipart["Subject"] = subject  
#         attachment = MIMEApplication(df.to_csv())
#         attachment["Content-Disposition"] = 'attachment; filename=" {}"'.format(f"{subject}.csv")
#         multipart.attach(attachment)
#         multipart.attach(MIMEText(message, "html"))
#         server = smtplib.SMTP("smtp.gmail.com", 587)
#         server.starttls()
#         server.login(multipart["From"], password)
#         server.sendmail(multipart["From"], multipart["To"], multipart.as_string())
#         server.quit()

In [137]:
#TEAMS
import requests

link = BaseURL+'/v1/standings/now'
f = requests.get(link)

i=0
teamlist=[]
abbrevlist=[]
citylist=[]

while ((len(teamlist)<32)|(len(teamlist)<32)):
    try:
        citylist.append(f.json()['standings'][i]['placeName']['default'])
        teamlist.append(f.json()['standings'][i]['teamCommonName']['default'])
        abbrevlist.append(f.json()['standings'][i]['teamAbbrev']['default'])
        #teamlist=list(set(teamlist))
        i=i+1
    except:
        break
len(teamlist)

Teams=pd.DataFrame()
Teams['City']=citylist
Teams['TeamName']=teamlist
Teams['TeamID']=abbrevlist
Teams

,City,TeamName,TeamID
0,New Jersey,Devils,NJD
1,Utah,Mammoth,UTA
2,Pittsburgh,Penguins,PIT
3,Colorado,Avalanche,COL
4,Montréal,Canadiens,MTL
5,Vegas,Golden Knights,VGK
6,Carolina,Hurricanes,CAR
7,Winnipeg,Jets,WPG
8,Washington,Capitals,WSH
9,Detroit,Red Wings,DET


In [138]:
STARTINGCSV.head()

,playerId,sweaterNumber,name,position,goals,assists,points,plusMinus,pim,hits,powerPlayGoals,sog,faceoffWinningPctg,toi,blockedShots,shifts,giveaways,takeaways,firstName,lastName,Team,gameId,homeRoadFlag
0,8477987,8,{'default': 'R. Donato'},C,0,0,0,0,0,3,0,1,0.000000,16:00,0,23,0,0,Ryan,Donato,CHI,2025020001,0
1,8477450,16,{'default': 'J. Dickinson'},C,0,0,0,0,0,0,0,1,0.578947,14:14,0,25,0,0,Jason,Dickinson,CHI,2025020001,0
2,8473422,17,{'default': 'N. Foligno'},L,0,0,0,-1,5,2,0,0,0.625000,11:48,0,21,2,0,Nick,Foligno,CHI,2025020001,0
3,8483450,20,{'default': 'R. Greene'},C,0,0,0,-1,0,0,0,0,0.250000,08:17,0,15,0,0,Ryan,Greene,CHI,2025020001,0
4,8478043,24,{'default': 'S. Lafferty'},C,0,0,0,-1,0,1,0,0,0.000000,08:07,0,13,0,0,Sam,Lafferty,CHI,2025020001,0


In [139]:
playerStats=STARTINGCSV[['Team','playerId','firstName','lastName','gameId','homeRoadFlag','sog','toi','position']]
playerStats['toi']=round(pd.to_numeric(playerStats['toi'].str.slice(0,2).str.replace(':', '', regex=False)
                                 )+(pd.to_numeric(playerStats['toi'].str.slice(3, 5))/60),2) #min+sec in decimal base not base60
playerStats['GameNumber'] = playerStats.groupby(['playerId'])['gameId'].cumcount(ascending=False)+1
playerStats['GP']=playerStats.groupby('Team').GameNumber.transform(np.max)
playerStats['HomeGames'] = playerStats.groupby(['playerId']).homeRoadFlag.transform(np.sum)
playerStats['AwayGames']=playerStats['GP']-playerStats['HomeGames']
playerStats['TotalShotsSzn']=playerStats.groupby(['playerId']).sog.transform(np.sum)
playerStats['SeasonAvgSOG']= round(playerStats['TotalShotsSzn']/playerStats['GP'],2)
playerStats['AvgShotsSznByLoc']=playerStats.groupby(['playerId','homeRoadFlag']).sog.transform(np.mean)

playerStats['LastFiveFlag'] = np.where(playerStats['GameNumber'] >= playerStats['GP']-5, 1, 0)
playerStats['LastFiveAvgShots']=playerStats.groupby(['playerId','LastFiveFlag']).sog.transform(np.sum)/5
playerStats['LastFiveAvgShots']=np.where(playerStats['LastFiveFlag'] == 1, playerStats['LastFiveAvgShots'], 0)
playerStats['LastFiveAvgShots']=playerStats.groupby('playerId').LastFiveAvgShots.transform(np.max)

playerStats['LastTenFlag'] = np.where(playerStats['GameNumber'] >= playerStats['GP']-10, 1, 0)
playerStats['LastTenAvgShots']=playerStats.groupby(['playerId','LastTenFlag']).sog.transform(np.sum)/10
playerStats['LastTenAvgShots']=np.where(playerStats['LastTenFlag'] == 1, playerStats['LastTenAvgShots'], 0)
playerStats['LastTenAvgShots']=playerStats.groupby('playerId').LastTenAvgShots.transform(np.max)

playerStats['avgshotshomevaway']=round(playerStats.groupby(['playerId','homeRoadFlag']).sog.transform(np.mean),2)


playerStats[playerStats['homeRoadFlag']==1]


,Team,playerId,firstName,lastName,gameId,homeRoadFlag,sog,toi,position,GameNumber,GP,HomeGames,AwayGames,TotalShotsSzn,SeasonAvgSOG,AvgShotsSznByLoc,LastFiveFlag,LastFiveAvgShots,LastTenFlag,LastTenAvgShots,avgshotshomevaway
12,FLA,8477935,Sam,Bennett,2025020001,1,4,19.25,C,10,10,5,5,21,2.1,2.200000,1,3.0,1,2.1,2.20
13,FLA,8478421,A.J.,Greer,2025020001,1,1,7.32,L,10,10,5,5,6,0.6,0.800000,1,0.4,1,0.6,0.80
14,FLA,8482713,Mackie,Samoskevich,2025020001,1,0,11.30,R,10,10,5,5,26,2.6,2.000000,1,2.4,1,2.6,2.00
15,FLA,8479981,Jonah,Gadjovich,2025020001,1,1,7.30,L,10,10,5,5,8,0.8,1.200000,1,0.6,1,0.8,1.20
16,FLA,8477933,Sam,Reinhart,2025020001,1,3,18.23,C,10,10,5,5,26,2.6,3.400000,1,3.0,1,2.6,3.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5431,OTT,8475324,Nick,Jensen,2025020149,1,1,18.75,D,1,10,6,4,8,0.8,0.500000,0,0.8,1,0.8,0.50
5432,OTT,8484321,Nikolas,Matinpalo,2025020149,1,0,17.52,D,1,10,3,7,3,0.3,0.666667,0,0.2,1,0.3,0.67
5433,OTT,8482095,Tyler,Kleven,2025020149,1,0,19.35,D,1,10,6,4,9,0.9,0.833333,0,1.4,1,0.9,0.83
5434,OTT,8478469,Thomas,Chabot,2025020149,1,1,20.33,D,1,10,6,4,14,1.4,1.000000,0,1.6,1,1.4,1.00


In [140]:
res = playerStats.groupby(['playerId'])["sog"].quantile([0.05, 0.95]).unstack(level=1)
res.reset_index(inplace=True)
res=res.set_axis(['playerId','Low','High'], axis=1)
res=playerStats.merge(res,on=['playerId'],how='left')
ShotsNoOutliers=res[(res['Low']<res['sog'])&(res['sog']<res['High'])].drop(columns=['Low', 'High'])

#toi
res = playerStats.groupby(['playerId'])["toi"].quantile([0.05, 0.95]).unstack(level=1)
res.reset_index(inplace=True)
res=res.set_axis(['playerId','Low','High'], axis=1)
res=ShotsNoOutliers.merge(res,on=['playerId'],how='left')
ShotsNoOutliers=res[(res['Low']<res['toi'])&(res['toi']<res['High'])].drop(columns=['Low', 'High'])

ShotsNoOutliers['Avgtoi']=ShotsNoOutliers.groupby(['playerId']).toi.transform(np.mean)

ShotsNoOutliers['GameNumberShotsNoOut'] = ShotsNoOutliers.groupby(['playerId'])['sog'].cumcount(ascending=False)+1
ShotsNoOutliers['GPShotsNoOut']=ShotsNoOutliers.groupby(['playerId']).GameNumberShotsNoOut.transform(np.max)
ShotsNoOutliers['NoOutliersAvgSOG']= ShotsNoOutliers.groupby(['playerId']).sog.transform(np.sum)/ShotsNoOutliers['GPShotsNoOut']
ShotsNoOutliers=ShotsNoOutliers[ShotsNoOutliers['NoOutliersAvgSOG']>1.3]
ShotsNoOutliers

,Team,playerId,firstName,lastName,gameId,homeRoadFlag,sog,toi,position,GameNumber,GP,HomeGames,AwayGames,TotalShotsSzn,SeasonAvgSOG,AvgShotsSznByLoc,LastFiveFlag,LastFiveAvgShots,LastTenFlag,LastTenAvgShots,avgshotshomevaway,Avgtoi,GameNumberShotsNoOut,GPShotsNoOut,NoOutliersAvgSOG
0,CHI,8477987,Ryan,Donato,2025020001,0,1,16.00,C,9,9,5,4,22,2.44,2.250000,1,3.0,1,2.2,2.25,15.791667,6,6,2.333333
1,CHI,8477450,Jason,Dickinson,2025020001,0,1,14.23,C,6,9,3,6,8,0.89,1.000000,1,0.8,1,0.8,1.00,15.260000,3,3,1.333333
5,CHI,8483493,Frank,Nazar,2025020001,0,1,20.23,C,9,9,5,4,16,1.78,2.500000,1,1.6,1,1.6,2.50,20.016667,3,3,2.000000
6,CHI,8481624,Ilya,Mikheyev,2025020001,0,1,17.12,R,9,9,5,4,19,2.11,1.500000,1,2.8,1,1.9,1.50,17.260000,5,5,2.200000
7,CHI,8484144,Connor,Bedard,2025020001,0,2,22.00,C,9,9,5,4,25,2.78,2.000000,1,2.0,1,2.5,2.00,21.612000,5,5,2.200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2671,OTT,8480188,Fabian,Zetterlund,2025020149,1,1,14.85,L,1,10,6,4,18,1.80,1.333333,0,2.0,1,1.8,1.33,13.952000,1,5,1.800000
2672,OTT,8473512,Claude,Giroux,2025020149,1,3,17.62,R,1,10,6,4,16,1.60,1.666667,0,1.8,1,1.6,1.67,16.256000,1,5,2.000000
2673,OTT,8474102,David,Perron,2025020149,1,1,14.47,L,1,10,6,4,13,1.30,1.500000,0,1.0,1,1.3,1.50,14.408000,1,5,1.400000
2674,OTT,8482092,Ridly,Greig,2025020149,1,2,15.90,C,1,10,6,4,14,1.40,1.500000,0,2.0,1,1.4,1.50,17.036000,1,5,1.600000


In [141]:
import datetime
# todaysindex=datetime.datetime.today().weekday()+2
# if  todaysindex>6:
#     todaysindex-=7
#I think todaysindex is always 3
todaysindex=3

TodaysGames=pd.DataFrame()
link = BaseURL + '/v1/scoreboard/now' 
f = requests.get(link)
df=pd.DataFrame(f.json()['gamesByDate'][todaysindex]['games'])

for i in range(0,len(df)):
    link = BaseURL + '/v1/scoreboard/now' 
    f = requests.get(link)
    AwayTeam=f.json()['gamesByDate'][todaysindex]['games'][i]['awayTeam']['abbrev']
    HomeTeam=f.json()['gamesByDate'][todaysindex]['games'][i]['homeTeam']['abbrev']

    dict = {'AwayTeam':[AwayTeam], 
        'HomeTeam':[HomeTeam]} 

    ThisGame=pd.DataFrame(dict)
    TodaysGames=pd.concat([TodaysGames,ThisGame], ignore_index = True)

TodaysGames


,AwayTeam,HomeTeam
0,PIT,PHI
1,CGY,TOR
2,VGK,CAR
3,CBJ,BUF
4,ANA,FLA
5,NYI,BOS
6,TBL,NSH
7,WPG,MIN
8,DET,STL
9,WSH,DAL


In [142]:
shotSummary=ShotsNoOutliers[['Team', 'playerId','firstName', 'lastName', 'Avgtoi','position','SeasonAvgSOG', 'LastFiveAvgShots',
'LastTenAvgShots', 'NoOutliersAvgSOG','avgshotshomevaway']].drop_duplicates()

teamsPlayingToday=TodaysGames['AwayTeam'].tolist()+TodaysGames['HomeTeam'].tolist()
allgames=pd.concat([TodaysGames,TodaysGames[['HomeTeam','AwayTeam']].rename(
    columns={"AwayTeam": "HomeTeam","HomeTeam": "AwayTeam"})], ignore_index = True)

todaysSkaters=shotSummary[shotSummary['Team'].isin(teamsPlayingToday)]

todaysSkaters=todaysSkaters.merge(allgames.rename(columns={"AwayTeam": "Opponent"})
                                  , left_on='Team', right_on='HomeTeam', how='left').drop(columns=['HomeTeam'])
todaysSkaters['HomeFlag'] = np.where(todaysSkaters['Team'].isin(TodaysGames['HomeTeam'].tolist()), 1, 0)

todaysSkaters.sort_values(by=['NoOutliersAvgSOG']
                        , ascending=False #or True for players trending down
                       ).reset_index(drop=True).head(20)

,Team,playerId,firstName,lastName,Avgtoi,position,SeasonAvgSOG,LastFiveAvgShots,LastTenAvgShots,NoOutliersAvgSOG,avgshotshomevaway,Opponent,HomeFlag
0,ANA,8483445,Cutter,Gauthier,18.215000,L,4.75,5.6,3.8,5.250000,4.00,FLA,0
1,ANA,8483445,Cutter,Gauthier,18.215000,L,4.75,5.6,3.8,5.250000,5.00,FLA,0
2,BUF,8479420,Tage,Thompson,19.563333,C,4.11,4.4,3.7,4.500000,3.50,CBJ,1
3,BUF,8479420,Tage,Thompson,19.563333,C,4.11,4.4,3.7,4.500000,5.33,CBJ,1
4,NJD,8481559,Jack,Hughes,20.216667,C,4.00,5.2,3.6,4.333333,4.40,COL,0
5,NJD,8481559,Jack,Hughes,20.216667,C,4.00,5.2,3.6,4.333333,3.50,COL,0
6,VGK,8478403,Jack,Eichel,21.170000,C,4.22,6.0,3.8,4.166667,5.25,CAR,0
7,DAL,8480027,Jason,Robertson,20.546667,L,4.33,5.2,3.9,4.166667,4.75,WSH,1
8,DAL,8480027,Jason,Robertson,20.546667,L,4.33,5.2,3.9,4.166667,4.00,WSH,1
9,VGK,8478403,Jack,Eichel,21.170000,C,4.22,6.0,3.8,4.166667,3.40,CAR,0


In [143]:

goalieSTARTINGCSV2025=pd.read_csv('NewGoalieStats2526.csv')  
goalieSTARTINGCSV=goalieSTARTINGCSV2025
goalieSTARTINGCSV.sort_values(by=['gameId'])

,playerId,sweaterNumber,name,position,evenStrengthShotsAgainst,powerPlayShotsAgainst,shorthandedShotsAgainst,saveShotsAgainst,savePctg,evenStrengthGoalsAgainst,powerPlayGoalsAgainst,shorthandedGoalsAgainst,pim,goalsAgainst,toi,starter,decision,shotsAgainst,saves,firstName,lastName,Team,gameId,homeRoadFlag
0,8481519,30,{'default': 'S. Knight'},G,29/31,4/5,1/1,34/37,0.918919,2,1,0,0,3,59:04,True,L,37,34,Spencer,Knight,CHI,2025020001,0
1,8475683,72,"{'default': 'S. Bobrovsky', 'cs': 'S. Bobrovsk...",G,15/17,2/2,0/0,17/19,0.894737,2,0,0,0,2,60:00,True,W,19,17,Sergei,Bobrovsky,FLA,2025020001,1
2,8481668,37,"{'default': 'A. Silovs', 'cs': 'A. Šilovs', 's...",G,22/22,3/3,0/0,25/25,1.000000,0,0,0,0,0,60:00,True,W,25,25,Arturs,Silovs,PIT,2025020002,0
3,8478048,31,"{'default': 'I. Shesterkin', 'cs': 'I. Šesťork...",G,28/29,0/0,0/0,28/29,0.965517,1,0,0,0,1,59:37,True,L,29,28,Igor,Shesterkin,NYR,2025020002,1
4,8475809,41,{'default': 'S. Wedgewood'},G,17/17,7/8,0/0,24/25,0.960000,0,1,0,0,1,59:53,True,W,25,24,Scott,Wedgewood,COL,2025020003,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,8477967,35,{'default': 'T. Demko'},G,25/27,1/2,0/0,26/29,0.896552,2,1,0,0,3,61:43,True,W,29,26,Thatcher,Demko,VAN,2025020147,1
298,8480981,30,{'default': 'J. Hofer'},G,20/25,0/0,0/0,20/25,0.800000,5,0,0,0,5,59:53,True,L,25,20,Joel,Hofer,STL,2025020148,0
299,8477465,35,{'default': 'T. Jarry'},G,21/24,4/4,0/0,25/28,0.892857,3,0,0,0,3,59:15,True,W,28,25,Tristan,Jarry,PIT,2025020148,1
300,8480280,1,{'default': 'J. Swayman'},G,14/17,1/5,1/1,16/23,0.695652,3,4,0,0,7,59:57,True,L,23,16,Jeremy,Swayman,BOS,2025020149,0


In [144]:
goalieStats=goalieSTARTINGCSV[['Team','playerId','firstName','lastName','gameId','starter','shotsAgainst','goalsAgainst','savePctg']]

goalieStats['PlayerGameNumber'] = goalieStats.groupby(['playerId'])['gameId'].cumcount(ascending=False)+1
goalieStats['TeamGameNumber'] = goalieStats.groupby(['Team'])['gameId'].cumcount(ascending=False)+1
goalieStats['PlayerGP']=goalieStats.groupby(['playerId']).starter.transform(np.sum)
goalieStats['TeamGP']=goalieStats.groupby(['Team']).starter.transform(np.sum)

goalieStats['AvgShotsAgainst']=goalieStats.groupby(['Team']).shotsAgainst.transform(np.sum)/goalieStats['TeamGP']
goalieStats['RankAvgShotsAgainst']=round(goalieStats['AvgShotsAgainst'].rank(pct=True)*100,2)

goalieStats['AvgGoalsAgainst']=goalieStats.groupby(['playerId']).goalsAgainst.transform(np.sum)/goalieStats['PlayerGP']
goalieStats['RankAvgGoalsAgainst']=round(goalieStats['AvgGoalsAgainst'].rank(pct=True)*100,2)

goalieStats['AvgSavePctg']=goalieStats.groupby(['playerId']).savePctg.transform(np.sum)/goalieStats['PlayerGP']
goalieStats['RankAvgSavePctg']=round(goalieStats['AvgSavePctg'].rank(pct=True)*100,2)

goalieStats['LastFiveFlag'] = np.where(goalieStats['TeamGameNumber'] >= goalieStats['TeamGP']-5, 1, 0)

goalieStats['LastFiveAvgShotsAgainst']=goalieStats.groupby(['playerId','LastFiveFlag']).shotsAgainst.transform(np.sum)/5
goalieStats['LastFiveAvgShotsAgainst']=np.where(goalieStats['LastFiveFlag'] == 1, goalieStats['LastFiveAvgShotsAgainst'], 0)
goalieStats['LastFiveAvgShotsAgainst']=goalieStats.groupby('playerId').LastFiveAvgShotsAgainst.transform(np.max)

summaryGoalieStats=goalieStats[['Team','PlayerGP','firstName','lastName','RankAvgShotsAgainst','RankAvgGoalsAgainst','RankAvgSavePctg']].drop_duplicates()

summaryGoalieStats['GPRank']=summaryGoalieStats.groupby(['Team'])['PlayerGP'].rank(ascending=False).astype(int)
summaryGoalieStats=summaryGoalieStats[['Team','GPRank','firstName','lastName','RankAvgShotsAgainst','RankAvgGoalsAgainst','RankAvgSavePctg']].drop_duplicates()

summaryGoalieStats=summaryGoalieStats.rename(columns={"firstName": "GoalieFirstName",'lastName':'GoalieLastName','Team':'GoalieteamAbbrev'})
summaryGoalieStats.head()

,GoalieteamAbbrev,GPRank,GoalieFirstName,GoalieLastName,RankAvgShotsAgainst,RankAvgGoalsAgainst,RankAvgSavePctg
0,CHI,1,Spencer,Knight,72.19,11.75,96.19
1,FLA,1,Sergei,Bobrovsky,14.40,24.67,57.12
2,PIT,1,Arturs,Silovs,89.90,40.56,80.13
3,NYR,1,Igor,Shesterkin,27.65,24.67,75.00
4,COL,1,Scott,Wedgewood,7.78,19.21,37.09


In [145]:
full=todaysSkaters.merge(summaryGoalieStats
                         [summaryGoalieStats['GPRank']==1]
                         , left_on='Opponent', right_on='GoalieteamAbbrev', how='left'
                         ).drop(columns=['GoalieteamAbbrev','GPRank'])
full['lastfivemargin']=full['LastFiveAvgShots']-full['NoOutliersAvgSOG']

full['locationmargin']=full['avgshotshomevaway']-full['NoOutliersAvgSOG']
full['heatermargin']=full['LastFiveAvgShots']-full['LastTenAvgShots']
full['fixedmargin']=full['lastfivemargin']-full['heatermargin']
full['goalieweight']=2*((full['RankAvgShotsAgainst']-50)/100)
full['combinedmargin']=full['locationmargin']+full['fixedmargin']+full['goalieweight']

full['A_Standardized'] = (full['combinedmargin'] - full['combinedmargin'].mean()) / (full['combinedmargin'].std())

full['overrated'] = np.where(full['A_Standardized']<0,full['NoOutliersAvgSOG']-abs(full['A_Standardized']),
abs(full['A_Standardized'])-full['NoOutliersAvgSOG'])

full=full[abs(full['overrated'])>0.79]

full['aboveexpected']=full['overrated']/full['NoOutliersAvgSOG']

full=full[['lastName','firstName','Team', 'NoOutliersAvgSOG','combinedmargin'
,'A_Standardized','overrated','aboveexpected','fixedmargin','goalieweight',
'Avgtoi','position', 'HomeFlag','Opponent','heatermargin',  'LastFiveAvgShots', 'LastTenAvgShots',
'lastfivemargin','avgshotshomevaway','locationmargin','RankAvgShotsAgainst'
]].drop_duplicates().rename(columns={"NoOutliersAvgSOG": "ExpectedSOG",
        "avgshotshomevaway": "avgshotbyLoc"}).round(2)


todaysdate=str(pd.Timestamp.now().date())
full.to_excel(todaysdate+'skaterstats.xlsx', index=False, engine='xlsxwriter')

In [146]:
#full.insert(loc=1, column='new_col', value=[str('test') for i in range(0,len(full))])
print(len(full))
full.sort_values(by=['aboveexpected','overrated']
    #by=['ExpectedSOG']
                        , ascending=False #True #False #or True for players trending down
                       ).reset_index(drop=True).head(25)

497


,lastName,firstName,Team,ExpectedSOG,combinedmargin,A_Standardized,overrated,aboveexpected,fixedmargin,goalieweight,Avgtoi,position,HomeFlag,Opponent,heatermargin,LastFiveAvgShots,LastTenAvgShots,lastfivemargin,avgshotbyLoc,locationmargin,RankAvgShotsAgainst
0,Stützle,Tim,OTT,3.50,-0.29,-0.01,3.49,1.00,-0.40,0.44,18.98,C,0,CHI,1.1,4.2,3.1,0.70,3.17,-0.33,72.19
1,Giroux,Claude,OTT,2.00,-0.29,-0.01,1.99,1.00,-0.40,0.44,16.26,R,0,CHI,0.2,1.8,1.6,-0.20,1.67,-0.33,72.19
2,Toews,Devon,COL,1.60,-0.28,-0.00,1.60,1.00,-0.10,-0.58,24.15,D,1,NJD,0.1,1.6,1.5,0.00,2.00,0.40,21.19
3,Connor,Kyle,WPG,2.75,-0.30,-0.03,2.72,0.99,0.55,0.57,20.42,L,0,MIN,0.3,3.6,3.3,0.85,1.33,-1.42,78.31
4,Rielly,Morgan,TOR,2.67,-0.31,-0.03,2.63,0.99,-0.57,0.21,21.92,D,1,CGY,0.7,2.8,2.1,0.13,2.71,0.04,60.60
5,Seider,Moritz,DET,2.25,-0.31,-0.03,2.22,0.99,-0.15,-0.91,25.50,D,0,STL,0.3,2.4,2.1,0.15,3.00,0.75,4.64
6,Smith,Cole,NSH,1.60,-0.30,-0.02,1.58,0.99,-0.20,0.50,14.93,R,1,TBL,-0.2,1.2,1.4,-0.40,1.00,-0.60,75.17
7,Lehkonen,Artturi,COL,2.83,-0.34,-0.06,2.77,0.98,-0.43,-0.58,19.64,L,1,NJD,0.4,2.8,2.4,-0.03,3.50,0.67,21.19
8,Cuylle,Will,NYR,2.50,-0.32,-0.04,2.46,0.98,-0.40,0.38,17.31,L,0,VAN,1.3,3.4,2.1,0.90,2.20,-0.30,69.04
9,Kreider,Chris,ANA,2.00,-0.31,-0.03,1.97,0.98,-0.60,-0.71,17.33,L,0,FLA,0.6,2.0,1.4,0.00,3.00,1.00,14.40


In [147]:
# pip install xlsxwriter

In [148]:
# margin=full
# margin['lastfivemargin']=margin['LastFiveAvgShots']-margin['ExpectedSOG']


# margin=margin[['teamAbbrev', 'HomeFlag', 'firstName', 'lastName', 'ExpectedSOG','AvgShotsSznByLoc',
#        'LastFiveAvgShots', 'LastTenAvgShots', 'Opponent', 'RankAvgShotsAgainst',
#         'GoalieLastName',	'avgshotsvsOppAtLoc',	'AvgShotsAgainstThisOpp',
#         'SeasonAvgSOG', 'playerId', 
#        'RankAvgGoalsAgainst', 'RankAvgSavePctg', 'lastfivemargin']]

# margin.sort_values(by=['lastfivemargin']
#                         , ascending=False #or True for players trending down
#                         #, ascending=True
#                        ).reset_index(drop=True).head(25)

In [149]:
# margin['locationmargin']=margin['AvgShotsSznByLoc']-margin['ExpectedSOG']

# margin.sort_values(by=['locationmargin']
#                         , ascending=False #or True for players trending down
#                         #, ascending=True
#                        ).reset_index(drop=True).head(25)


In [150]:
# margin['combinedmargin']=margin['lastfivemargin']+margin['locationmargin']

# margin[['firstName',	'lastName','teamAbbrev',	'HomeFlag','combinedmargin',
# 		'ExpectedSOG','LastFiveAvgShots',	'AvgShotsSznByLoc'	,
#         	'LastTenAvgShots',	'Opponent',	'RankAvgShotsAgainst',	'GoalieLastName',
#             			'SeasonAvgSOG']].sort_values(by=['combinedmargin']
#                         , ascending=False #or True for players trending down
#                         #, ascending=True
#                        ).reset_index(drop=True).head(25)

In [151]:
# link = BaseURL + '/v1/partner-game/US/now'
# f = requests.get(link)
# f.json()['games'][0]['homeTeam']
# # df=pd.DataFrame(f.json()['games'][0])
# # df